In [ ]:
## Takes a bit time to read the kaggle . json file 
!pip install kaggle --upgrade
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!mkdir content/data
!kaggle datasets download -d lyfeisgood/something
!unzip /content/something.zip -d data

In [18]:
import os
import re
import nibabel as nib
import numpy as np
import torch
print(torch.__version__)
print(torch.cuda.is_available())
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
import torch

1.12.0+cu113
True


In [19]:
def get_mri_data(path_mri):
    '''Returns a list containing the path of MRI DATA'''
    path = path_mri;
    fileRegex1 = re.compile(r'T1_2_std_warped'); mri_data_path = []; 
    # Iterating over Directories 
    for subdir, dirs, files in os.walk(path):
        for file in files:
            # Storing the path of Data
            if fileRegex1.search(file): mri_data_path.append(os.path.join(subdir, file));
    return mri_data_path;

def get_asl_data(path_asl):
    '''Returns a list containing the path of ASL DATA'''
    asl_path = path_asl; asl_data_path = []; 
    # Iterating over directories
    for subdir, dirs, files in os.walk(asl_path):
        # Selecting the ASL file
        for file in files:
            if file == 'perfusion.nii':
                asl_data_path.append(os.path.join(subdir, file));
    return asl_data_path;

def create_data_dict(mri_data_path,asl_data_path): 
    '''To create a Dictionary of MRI_data and ASL_data'''
    mri_map = {}; asl_map = {}; patient = re.compile(r'sub*'); 
    
    ## Iterating over the mri data's path
    for i in mri_data_path:
    ## Value is our path i.e. stored and key is the patient number
        value = i; key = i.split('/'); 
        for vals in key:
            if patient.search(vals): mri_map[vals.split('-')[1]] = value;
    ## Iterating over the mri data's path
    for i in asl_data_path:
        ## Value is our path i.e. stored and key is the patient number
        value = i; key = i.split('/'); 
        for vals in key:
            if patient.search(vals): asl_map[vals.split('-')[1]] = value;
    return mri_map,asl_map;

def create_tensors(mri_data_dict,asl_data_dict,device):
    '''Creates tensors from the data dictionary feeded'''
    x = [] ; y = []; X = []; Y = [];
    for k,v in mri_data_dict.items():
        if k in asl_data_dict:
            # Loading the MRI image from the path in the train x path 
            mri_img = nib.load(v); 
            # Making it a numpy array
            mri_vec = np.array(mri_img.dataobj)[np.newaxis,:,:,:] # Channels x Length X Breadth X Slices of Brain
            # Min max Normalizing the image 
            mri_vec = (mri_vec - mri_vec.min()) / (mri_vec.max() - mri_vec.min())
            # Appending the MRI image to X 
            x.append(torch.as_tensor(mri_vec,dtype=torch.float32).to(device)); 
            ## Finding the same patient with ASL data 
            asl_img = nib.load(asl_data_dict[k]); asl_vec = np.array(asl_img.dataobj)[np.newaxis,:,:,:]; 
            asl_vec = (asl_vec - asl_vec.min()) / (asl_vec.max() - asl_vec.min());
            ## Appending the image to y
            y.append(torch.as_tensor(asl_vec,dtype=torch.float32).to(device))
            X = torch.stack(x,dim=0).to(device); Y = torch.stack(y,dim=0).to(device);
    return X,Y

def print_data_shape(data):
    '''Prints the shape of data'''
    for i in data: print(i.shape);

def tensor_stats(tensor_array):
    '''Prints some statistics about the tensors'''
    for i in tensor_array:
        print(i.sum(), i.prod(), i.mean(), i.std());

def print_data_dimension(data):
    '''Print dimension of data'''
    print(data.shape)

def data_loader(path_mri,path_asl,device):
    '''This function will return us the mri and asl data in format we want'''
    ## Getting path of all MRI data stored and ASL data stored
    mri_data = get_mri_data(path_mri); asl_data = get_asl_data(path_asl);
    ## Making dict of MRI data for which ASL data exists
    mri_data_dict , asl_data_dict = create_data_dict(mri_data,asl_data);
    ## Creating the Tensors of the MRI and ASL data
    x , y = create_tensors(mri_data_dict,asl_data_dict,device);
    return x,y

def data_split(x,y,size):
    '''Split data into train and test'''
    train_x = x[0:int((len(x)+1)*(1-size))] ; train_y = y[0:int((len(x)+1)*(1-size))] ; 
    test_x = x[int((len(x)+1)*(1-size)):] ; test_y = y[int((len(x)+1)*(1-size)):];
    return train_x,train_y,test_x,test_y; 

x,y = data_loader('/content/data','/content/data',device)
## Printing the shape of specifid data
print('Shape of MRI images - >');       print_data_shape(x); 
print('\nShape of ASL images - >');     print_data_shape(y);

## Printing the Stats of the data
print('\nStats for MRI data(X) - >');   tensor_stats(x); 
print('\nStats for ASL data(X) - >');   tensor_stats(y);

## Printing the Dimension of X and Y  
## Dimension refer -> Number x Length x Breadth x Height x Channel
print('\nDimensions of X(MRI Data) ->',end=' ');print_data_dimension(x)

Shape of MRI images - >
torch.Size([1, 182, 218, 182])
torch.Size([1, 182, 218, 182])
torch.Size([1, 182, 218, 182])
torch.Size([1, 182, 218, 182])
torch.Size([1, 182, 218, 182])
torch.Size([1, 182, 218, 182])
torch.Size([1, 182, 218, 182])
torch.Size([1, 182, 218, 182])
torch.Size([1, 182, 218, 182])
torch.Size([1, 182, 218, 182])

Shape of ASL images - >
torch.Size([1, 91, 109, 91])
torch.Size([1, 91, 109, 91])
torch.Size([1, 91, 109, 91])
torch.Size([1, 91, 109, 91])
torch.Size([1, 91, 109, 91])
torch.Size([1, 91, 109, 91])
torch.Size([1, 91, 109, 91])
torch.Size([1, 91, 109, 91])
torch.Size([1, 91, 109, 91])
torch.Size([1, 91, 109, 91])

Stats for MRI data(X) - >
tensor(803922.6875, device='cuda:0') tensor(-0., device='cuda:0') tensor(0.1113, device='cuda:0') tensor(0.1415, device='cuda:0')
tensor(767731.0625, device='cuda:0') tensor(-0., device='cuda:0') tensor(0.1063, device='cuda:0') tensor(0.1298, device='cuda:0')
tensor(910082.3125, device='cuda:0') tensor(0., device='cuda:0')